# (some) Related Work

# Abstract

# Intro to Numba
In the next section we will be needing some speed to run some tests. Python isn't known for speed. We will for now use numba to speed up Python because it requires basically the no amount of effort if you already know Python just some details you have know. <br>

## What is Numba
Go see https://numba.pydata.org/. <br>
Numba translates Python functions to optimized machine code at runtime using the industry-standard LLVM compiler library. Numba-compiled numerical algorithms in Python can approach the speeds of C or FORTRAN.

You don't need to replace the Python interpreter, run a separate compilation step, or even have a C/C++ compiler installed. Just apply one of the Numba decorators to your Python function, and Numba does the rest. <br>

## Installation
We installed numba via anaconda (conda install numba). We think you can also install it with pip if you have the correct version of Python.

## Example
Here is a quick example to get your feet wet:

In [45]:
from random import random
from numba import njit, prange

# normal way:
def Y(t): return 1 + (Y(random()*t) if random()<t else 0)
def y(t,nsim): return sum(Y(t) for _ in range(nsim))/nsim

# numba way:
@njit(fastmath=True)
def Yn(t): return 1.0 + (Yn(random()*t) if random()<t else 0.0)

@njit(fastmath=True, parallel=True)
def yn(t, nsim):
    total = 0.0 # <- typing becomes important 0 vs 0.0
    for _ in prange(nsim):
        total += Yn(t)
    return total/nsim

In [49]:
t,nsim = 1,10**5
%timeit y(t,nsim)
%timeit yn(t,nsim)

215 ms ± 89.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
5.77 ms ± 295 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In this example numba speeded up the running time 40 times (on our laptop) with minor changes (typing, decorators). 

## Disadvantages
Here are some of the main drawbacks of using Numba:

-   Limited support for Python features: Numba supports a limited subset of Python features, which means that not all Python code can be accelerated with Numba.

-   Compilation time: Numba needs to compile the code before it can execute it, which can take some time. This overhead can be significant for small computations, making it less efficient than native Python.

-   Memory usage: Numba can increase the memory usage of Python programs, which can be a problem for large-scale computations.

-   Debugging: Numba can make debugging more difficult, as the compiled code is not as easy to read and understand as native Python code.